In [ ]:
%conda install matplotlib
%conda install python-dateutil
%conda install plotly
%conda install pandas
%conda install nbformat
%conda install pytz
%conda install requests

In [ ]:
import requests
from datetime import datetime, timedelta
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from requests.models import PreparedRequest
from urllib.request import Request, urlopen
import numpy as np
from dateutil.relativedelta import relativedelta
import json
import plotly.express as px
import pandas as pd
import pytz

In [ ]:
# You will be given a token to access the HCDP API. Add that token here.
hcdp_api_token = ""
# Please input your email address. This will be used for user logging or distributing data packages
email = ""

api_base_url = "https://api.hcdp.ikewai.org"
# Setup header for API requests
header = {
  "Authorization": f"Bearer {hcdp_api_token}"
}


In [ ]:
def display_raster(params, title, cmap = plt.cm.viridis.reversed(), nodata_color = "#f0f0f0"):
    #construct raster endpoint url base
    raster_ep = "/raster"
    url = f"{api_base_url}{raster_ep}"
    #construct url with params
    url_constructor = PreparedRequest()
    url_constructor.prepare_url(url, params)
    full_url = url_constructor.url
    print(f"Constructed API request URL: {full_url}")
    #create request object for use with urlopen
    req = Request(full_url, headers = header)
    #seupt plot
    fig, ax = plt.subplots(figsize=(20, 10), facecolor = "#e0e0e0")
    #remove axis ticks (displays row, col numbers, not super helpful)
    ax.axes.get_xaxis().set_ticks([])
    ax.axes.get_yaxis().set_ticks([])
    #set plot title
    plt.title(title, fontsize = 20)
    #set nodata value
    cmap.set_bad(nodata_color)
    #open data stream from API
    with urlopen(req) as raster:
        #read tiff image
        img = mpimg.imread(raster, format = "tiff")
        #mask nodata values
        masked = np.ma.masked_equal(img, img[0][0])
        #plot on map with color schema and add color bar
        imgplot = ax.imshow(masked[:, :, 0], cmap = cmap)
        fig.colorbar(imgplot, ax = ax)

In [ ]:
today = datetime.now(pytz.timezone("US/Hawaii") )
yesterday = today - timedelta(days = 1)
last_month = today - relativedelta(months = 1)
yesterday_str = yesterday.strftime("%Y-%m-%d")
last_month_str = last_month.strftime("%Y-%m")

params = {
    # The datatype parameter represents the variable of interest
    "datatype": "temperature",
    # The aggregation parameter is for temperature only and represents how the values were aggregated
    # Aggregation can be "min", "max", or "mean"
    "aggregation": "max",
    # Period can be "day" or "month" (the period over which measurements are taken over)
    "period": "day",
    "date": yesterday_str,
    # The extent of the data to be provided (statewide or county based)
    # Extent can be "statewide" (covers the entire state), "bi" (Hawaiʻi county), "mn" (Maui county), "oa" (Honolulu county), or "ka" (Kauaʻi county)
    "extent": "statewide"
}
display_raster(params, f"Maximum Temperature for {yesterday_str} (°C)", cmap = plt.cm.coolwarm)

params = {
    "datatype": "rainfall",
    # The production parameter is for rainfall only and represents the technique used to create the gridded map products
    # Production can be "new" or "legacy". Legacy rainfall maps are available from 1920-2012, whereas new rainfall maps are available from 1990-present
    "production": "new",
    # Gridded rainfall map products are currently only available for the new production technique up to 2019
    "period": "month",
    "date": last_month_str,
    "extent": "statewide"
}
display_raster(params, f"Hawaii Rainfall for {last_month_str} (mm)")

params = {
    "datatype": "relative_humidity",
    # Relative humidity data is currently only daily
    "period": "day",
    "date": yesterday_str,
    "extent": "bi"
}
display_raster(params, f"Big Island Relative Humidity for {yesterday_str} (%)")

# Feel free to play around with different datasets and extents



In [ ]:
def query_stations(values, name, limit = 10000, offset = 0):
    params = {
        "name": name
    }
    for key in values:
        params[f"value.{key}"] = values[key]
    params = {
        "q": json.dumps(params),
        "limit": limit,
        "offset": offset
    }

    stations_ep = "/stations"
    url = f"{api_base_url}{stations_ep}"

    res = requests.get(url, params, headers = header)
    res.raise_for_status()
    print(f"Constructed API request URL: {res.url}")
    res = [item["value"] for item in res.json()["result"]]
    return res

def get_station_metadata():
    res = query_stations({}, name = "hcdp_station_metadata")
    data = {}
    for metadata in res:
        data[metadata[metadata["id_field"]]] = metadata
    return data

def get_station_data(values, metadata = None, limit = 10000, offset = 0):
    res = query_stations(values, name = "hcdp_station_value", limit = limit, offset = offset)
    combined = res
    if metadata is not None:
        combined = []
        # combine values with metadata for station
        for item in res:
            station_metadata = metadata.get(item["station_id"])
            #only return data with metadata
            if station_metadata is not None:
                #combine item with metadata and add to combined array
                combined.append(item | station_metadata)
    return combined

In [ ]:
#map station rainfall data for last month

#retrieve station metadata
metadata = get_station_metadata()
#get all station data for last month
values = {
    "datatype": "rainfall",
    "production": "new",
    "period": "month",
    "fill": "partial",
    "date": last_month_str
}

data = get_station_data(values, metadata = metadata)
#set up dataframe from returned data
df_data = []
for item in data:
    df_data.append([float(item["lat"]), float(item["lng"]), round(float(item["value"]), 2)])
df = pd.DataFrame(df_data, columns = ["latitude", "longitude", "Rainfall (mm)"])
#map data values
fig = px.scatter_mapbox(df, lat = "latitude", lon = "longitude", size = "Rainfall (mm)", color = "Rainfall (mm)", color_continuous_scale = "rdbu", height = 950, width = 1600, zoom = 7)
fig.update_layout(mapbox_style="open-street-map")
fig.update_geos(fitbounds="locations")
fig.update_traces(marker_sizemin = 5)
fig.show()



In [ ]:
#create timeseries for the daily rainfall data from a single station

#pull first station's ID
station_id = data[0]["station_id"]

previous_year = today - relativedelta(years = 1)
previous_year_str = previous_year.strftime("%Y-%m-%d")

values = {
    "datatype": "rainfall",
    "production": "new",
    "period": "day",
    "fill": "partial",
    "station_id": station_id,
    "date": {
        "$gte": previous_year_str
    }
}
#get single station timeseries for the last year
data = get_station_data(values, metadata = metadata)
#construct dataframe from timeseries data
df_data = []
for item in data:
    df_data.append([item["value"], item["date"]])
df = pd.DataFrame(df_data, columns = ["Rainfall (mm)", "Date"])
df = df.sort_values(by = "Date")

fig = px.line(df, title = f"Last year of daily rainfall for station {station_id}", x = "Date", y = "Rainfall (mm)")
fig.show()

In [ ]:
#create a timeseries for maximum temperature data for an arbitrary map location

raster_timeseries_ep = "/raster/timeseries"
url = f"{api_base_url}{raster_timeseries_ep}"
#location of timeseries to get
lat = 19.7091
lng = -155.0954

params = {
    "datatype": "temperature",
    "aggregation": "max",
    "period": "day",
    "start": previous_year_str,
    "end": yesterday_str,
    "extent": "statewide",
    "lat": lat,
    "lng": lng
}
res = requests.get(url, params, headers = header)
res.raise_for_status()
print(f"Constructed API request URL: {res.url}")
data = res.json()
#construct dataframe from object (returned as JSON mapping of timestamps to values)
df_data = list(data.items())
df = pd.DataFrame(df_data, columns = ["Date", "Maximum Temperature (°C)"])
df = df.sort_values(by = "Date")

fig = px.line(df, title = f"Last year of daily rainfall for location Latitude: {lat}, Longitude: {lng}", x = "Date", y = "Maximum Temperature (°C)")
fig.show()


In [ ]:
#create data package and download immediately

genzip_content_ep = "/genzip/instant/content"
url = f"{api_base_url}{genzip_content_ep}"
#define data to be included in the dataset
#download last year of relative humidity data
data = [{
    "datatype": "relative_humidity",
    "period": "day",
    "extent": "statewide",
    "range": {
        "start": previous_year_str,
        "end": yesterday_str
    },
    "fill": "partial",
    #include station data csvs
    "files": ["station_data"]
},
#gridded map products and metadata files have slightly different parameters, so add to a second object
{
    "datatype": "relative_humidity",
    "period": "day",
    "extent": "statewide",
    "range": {
        "start": previous_year_str,
        "end": yesterday_str
    },
    #include gridded map data and metadata on how the maps were produced
    "files": ["data_map", "metadata"]
}]

params = {
    #add user email
    "email": email,
    "data": data
}

res = requests.post(url, json = params, headers = header)
print(f"Constructed API request URL: {res.url}, (params object is sent via the request body)")
res.raise_for_status()
#write the returned data to a file
with open("data.zip", "wb") as f:
    f.write(res.content)

print("Data written to local file data.zip")


In [ ]:
#create data package and send to email
#much better for large datasets

genzip_email_ep = "/genzip/email"
url = f"{api_base_url}{genzip_email_ep}"

res = requests.post(url, json = params, headers = header)
print(f"Constructed API request URL: {res.url}, (params object is sent via the request body)")
res.raise_for_status()
#print confirmation that the request is being processed
#on completion the package will be sent to your email
print(res.text)